In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import seaborn as sns
import os
import spacy
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
import bcolz 
import pickle
spacy_eng = spacy.load("en")
from tqdm import tqdm

In [2]:
def extract_data():
    dictionary = {}
    filenames = open('Dataset1/image_names.txt').read().strip().split('\n')
    for f in filenames:
        dictionary[f] = []
    captions = open('Dataset1/captions.txt').read().strip().split('\n')
    capt_map = [cap.strip().split('\t') for cap in captions]
    for i in range(len(capt_map)):
        capt_map[i][0] = capt_map[i][0][:-2]
        if capt_map[i][0] in dictionary:
            dictionary[capt_map[i][0]].append(capt_map[i][1])
    df = pd.DataFrame(dictionary.items(), columns = ['IMAGE', 'CAPTIONS'])
    df2 = pd.DataFrame(capt_map, columns =['IMAGE', 'CAPTION']) 
    df.to_csv('dataset_neat.csv',index = False)
    df2.to_csv('dataset_raw.csv',index = False)

In [3]:
extract_data()
df = pd.read_csv('dataset_neat.csv')
df2 = pd.read_csv('dataset_raw.csv')

In [4]:
class Specialised_Dataset(Dataset):
    def __init__(self,df,min_freq=2):
        
        self.min_freq = min_freq
        self.word2ind = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.ind2word = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.word2count = {"<PAD>": 0, "<SOS>": 0, "<EOS>": 0, "<UNK>": 0}  
        self.num_words = 4
        
        self.image_names = df['IMAGE']
        self.image_captions = df['CAPTION']
        self.caption_list = self.image_captions.to_list()
        self.build_vocabulary(self.caption_list)

        self.glove = self.create_embedding_map();
        
        
    
    def create_embedding_map(self):
        vectors = bcolz.open('6B.200.dat')[:]
        words = pickle.load(open('6B.200_words.pkl', 'rb'))
        word2idx = pickle.load(open('6B.200_idx.pkl', 'rb'))
        return {w: vectors[word2idx[w]] for w in words}
    
    
    def build_vocabulary(self,text_list):
        next_index = 4
        for sentence in text_list:
            for word in [tok.text.lower() for tok in spacy_eng.tokenizer(sentence)]:
                if word not in self.word2count:
                    self.word2count[word]=1
                else:
                    self.word2count[word]+=1
                if(self.word2count[word] == self.min_freq):
                    self.word2ind[word] = next_index
                    self.ind2word[next_index] = word
                    next_index+=1
        self.num_words = next_index
    
    def get_gLoVe_form(self,word): 
        if word in self.glove:
            return torch.tensor(self.glove[word])
        else:
            return torch.tensor(np.random.normal(scale=0.6, size=(200, )))

    def generate_gLoVe_matrix(self):
        gloVe_list = []
        for i in range(self.num_words):
            word = self.ind2word[i]
            gloVe_list.append(self.get_gLoVe_form(word))
        return torch.stack(gloVe_list)
    
        
    def numericalize(self,text):
        tokens = [tok.text.lower() for tok in spacy_eng.tokenizer(text)]
        return [
            self.word2ind[token] if token in self.word2ind else self.word2ind["<UNK>"]
            for token in tokens
        ]
    
    def __len__(self):
        return self.num_words
    
    def __getitem__(self,index):
        caption = self.caption_list[index]
        image_file = self.image_names[index]
        #print(Image)
        image = Image.open('Images/'+image_file).convert("RGB")
        #print(image)
        transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(),])
        image = transform(image)
        
        numericalized_caption = [self.word2ind["<SOS>"]]
        numericalized_caption += self.numericalize(caption)
        numericalized_caption.append(self.word2ind["<EOS>"])
        
        return image,torch.tensor(numericalized_caption)

In [5]:
class CustomPadderFunction:
    def __init__(self, padding):
        self.padding = padding

    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs, dim=0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first=False, padding_value=self.padding)
        return imgs, targets

In [6]:
Data = Specialised_Dataset(df2)
padding_index = Data.word2ind["<PAD>"]
#Hyperparams
batch_size = 10
shuffle = True
loader = DataLoader(dataset=Data,batch_size=batch_size,shuffle=shuffle,collate_fn=CustomPadderFunction(padding_index),)

In [16]:
class Convolutional_Encoder(nn.Module):
    def __init__(self,first_layer_depth,second_layer_depth,num_clusters,alpha=1.0,normalize_input=True):
        super().__init__()
        self.depth1 = first_layer_depth
        self.depth2 = second_layer_depth
        self.conv1 = nn.Conv2d(3,self.depth1,3,1)
        self.conv2 = nn.Conv2d(self.depth1,self.depth2,3,1)
        
        self.conv1.weight.data.fill_(0.1)
        self.conv2.weight.data.fill_(0.1)
        
        self.num_clusters = num_clusters
        self.alpha = alpha
        self.normalize_input = normalize_input
        
        self.netvlad_layer = nn.Conv2d(self.depth2, num_clusters, kernel_size=(1, 1), bias=True)
        self.centroids = nn.Parameter(torch.rand(self.num_clusters, self.depth2))
        
        self.netvlad_layer.weight = nn.Parameter(
            (2.0 * self.alpha * self.centroids).unsqueeze(-1).unsqueeze(-1)
        )
        self.netvlad_layer.bias = nn.Parameter(
            - self.alpha * self.centroids.norm(dim=1)
        )
        
    def forward(self,x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x,2,2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x,2,2)
        
        print('XOUT',x)
        
        N,C =  x.shape[:2]
        if self.normalize_input:
            x = nn.functional.normalize(x, p=2, dim=1)
            
        soft_assign = self.netvlad_layer(x).view(N, self.num_clusters, -1)
        soft_assign = nn.functional.softmax(soft_assign, dim=1)
        x_flatten = x.view(N, C, -1)
        residual = x_flatten.expand(self.num_clusters, -1, -1, -1).permute(1, 0, 2, 3) - \
            self.centroids.expand(x_flatten.size(-1), -1, -1).permute(1, 2, 0).unsqueeze(0)
        residual *= soft_assign.unsqueeze(2)
        vlad = residual.sum(dim=-1)

        vlad = nn.functional.normalize(vlad, p=2, dim=2)  # intra-normalization
        vlad = vlad.view(x.size(0), -1)  # flatten
        vlad = nn.functional.normalize(vlad, p=2, dim=1)  # L2 normalize
        
        return vlad

In [17]:
class RNN_Decoder(nn.Module):
    def __init__(self,embedding_size,hidden_size, input_size):
        super().__init__()
        self.embedding = nn.Embedding(input_size,embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, 1)
        self.fc = nn.Linear(hidden_size, input_size)
        
        for par in self.embedding.parameters():
            par.requires_grad = False
        
        
    def forward(self,image_features, captions):
        gLoVe_emb = self.embedding(captions)
        embeddings = torch.cat((image_features.unsqueeze(0), gLoVe_emb), dim=0)     
        hiddens, _ = self.lstm(embeddings)
        outputs = self.fc(hiddens)
        return outputs

In [18]:
class CNN_RNN_Network(nn.Module):
    def __init__(self, embed_size, hidden_size, input_size):
        super().__init__()
        self.encoder = Convolutional_Encoder(30,10,20)
        self.decoder = RNN_Decoder(embed_size,hidden_size,input_size)
    
    def forward(self,images,captions):
        features = self.encoder(images)
        #print(features.shape)
        outputs = self.decoder(features, captions)
        return outputs
    
    def generate_caption(self, image, dictionary, max_length = 30):
        caption = []
        
        with torch.no_grad():
            x = self.encoder(image).unsqueeze(0)
            #print(x)
            #states = None
            #for _ in range(max_length):
            #    #print('states',states)
            #    #print('Input',x)
            #    hiddens, states = self.decoder.lstm(x, states)
            ##    output = self.decoder.fc(hiddens.squeeze(0))
            #    pred = output.argmax(1)
            #    caption.append(pred.item())
            #    x = self.decoder.embedding(pred).unsqueeze(0)
            #    if(dictionary[pred.item()] == "<EOS>"):
            #        break
        
        return x
        #return [dictionary[idx] for idx in caption]

In [19]:
torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate = 0.1
num_epochs = 100

In [20]:
Net = CNN_RNN_Network(200,800,Data.num_words).to(device)
Net.decoder.embedding.weight.data.copy_(Data.generate_gLoVe_matrix().to(device))

tensor([[ 0.6850,  1.3936,  0.5608,  ...,  0.2518, -1.1372, -0.2139],
        [-0.1817,  0.0833, -0.1953,  ...,  0.8463,  0.6671, -0.4662],
        [ 0.4585,  0.7632,  0.4686,  ..., -0.2959, -0.3993, -0.6911],
        ...,
        [-0.2696,  0.2943, -0.5669,  ..., -0.7939, -0.4814, -0.1525],
        [-0.4282,  0.3101, -0.1896,  ...,  0.1711, -0.3179, -0.8590],
        [ 0.0837,  0.4184,  0.0520,  ..., -0.0124, -0.0672,  0.0180]],
       device='cuda:0')

In [21]:
files = ['test1.jpg','test2.jpg','test3.jpg','test4.jpg','test5.jpg']
def test():
    transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(),])
    for f in files:
        test_img = transform(Image.open(f).convert("RGB")).unsqueeze(0)
        Net.generate_caption(test_img.to(device), Data.ind2word)
        #print(test_img)
        print("Example 1 OUTPUT: "+ " ".join(Net.generate_caption(test_img.to(device), Data.ind2word)))

In [13]:

criterion = nn.CrossEntropyLoss(ignore_index=Data.word2ind["<PAD>"])
optimizer = torch.optim.Adam(Net.parameters(), lr=learning_rate)

for e in range(num_epochs):
    epoch_loss = 0
    for idx, (imgs, captions) in tqdm(enumerate(loader), total=len(loader), leave=False):
        imgs = imgs.to(device)
        captions = captions.to(device)
        outputs = Net(imgs, captions[:-1])
        loss = criterion(outputs.reshape(-1, outputs.shape[2]), captions.reshape(-1))
        optimizer.zero_grad()
        epoch_loss += loss.item()
        loss.backward(loss)
        optimizer.step()
    test()
    print(epoch_loss)
    #print(Net.parameters())

  0%|          | 2/517 [00:00<01:27,  5.86it/s]

tensor([[[[46.7839, 46.7333, 46.7663,  ..., 46.0651, 45.9816, 45.9227],
          [47.5992, 47.0710, 46.7533,  ..., 46.1345, 46.0721, 46.0639],
          [53.3086, 52.0286, 50.9886,  ..., 46.2545, 46.2321, 46.1345],
          ...,
          [31.6427, 31.6439, 31.5698,  ..., 31.1510, 30.8333, 30.3874],
          [30.7616, 30.6721, 30.3392,  ..., 29.2333, 29.0804, 29.3204],
          [30.0721, 30.0098, 29.6333,  ..., 29.3439, 29.8416, 30.5004]],

         [[46.7009, 46.6503, 46.6832,  ..., 45.9820, 45.8985, 45.8397],
          [47.5161, 46.9879, 46.6703,  ..., 46.0515, 45.9891, 45.9808],
          [53.2256, 51.9456, 50.9056,  ..., 46.1714, 46.1491, 46.0514],
          ...,
          [31.5597, 31.5609, 31.4867,  ..., 31.0679, 30.7503, 30.3044],
          [30.6785, 30.5891, 30.2562,  ..., 29.1503, 28.9973, 29.2373],
          [29.9891, 29.9267, 29.5503,  ..., 29.2609, 29.7585, 30.4173]],

         [[46.7438, 46.6932, 46.7261,  ..., 46.0250, 45.9414, 45.8826],
          [47.5591, 47.0308, 4

  1%|          | 4/517 [00:00<01:14,  6.87it/s]

tensor([[[[ 79.8479,  81.0595,  82.0020,  ...,  74.2340,  73.7117,  72.8570],
          [ 81.3372,  82.4960,  83.3104,  ...,  75.3832,  74.9951,  74.4942],
          [ 82.7828,  83.8281,  84.3964,  ...,  76.4662,  76.1341,  75.7593],
          ...,
          [ 50.0972,  48.1363,  49.8265,  ...,  73.6613,  76.0138,  78.0457],
          [ 37.9003,  43.0924,  43.0786,  ...,  74.9698,  70.8300,  67.6509],
          [ 42.4154,  45.5787,  44.5348,  ...,  67.9991,  62.3145,  58.4417]],

         [[326.5729, 331.5247, 335.3774,  ..., 303.6276, 301.4930, 298.0014],
          [332.6607, 337.3976, 340.7243,  ..., 308.3240, 306.7382, 304.6923],
          [338.5678, 342.8396, 345.1629,  ..., 312.7503, 311.3931, 309.8616],
          ...,
          [204.9155, 196.9099, 203.8044,  ..., 301.2950, 310.8863, 319.1758],
          [155.1642, 176.3722, 176.3124,  ..., 306.6331, 289.6865, 276.6510],
          [173.5264, 186.4922, 182.2363,  ..., 278.0844, 254.8792, 239.0671]],

         [[318.3563, 323.1825,

  1%|          | 6/517 [00:00<01:09,  7.38it/s]

...

 23%|██▎       | 120/517 [00:11<00:35, 11.24it/s]

tensor([[[[   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          ...,
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000]],

         [[ 969.0964, 1057.5695,  998.1807,  ...,  564.2018,  517.9747,
            463.6769],
          [1139.8329, 1160.5111, 1174.7510,  ...,  544.8013,  518.3967,
            466.5053],
          [ 996.1653, 1112.7216, 1151.8230,  ...,  484.7108,  468.5808,
            414.7408],
          ...,
          [ 930.5153,  971.6259,  971.3088,  ..., 1199.2611, 1233.4087,
           1222.3608],
          [ 925.00

 24%|██▎       | 122/517 [00:11<00:34, 11.31it/s]

tensor([[[[   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          ...,
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000]],

         [[1157.0194,  955.6046,  549.5120,  ...,  945.5104,  933.7246,
            861.3597],
          [ 561.2236,  406.8064,  507.5593,  ...,  836.4211,  852.9457,
            846.5664],
          [ 457.9375,  337.3545,  534.1287,  ...,  801.3091,  802.2013,
            805.3586],
          ...,
          [ 177.6912,  249.0210,  320.1911,  ...,  580.9341,  390.3537,
            306.3602],
          [ 224.17

KeyboardInterrupt: 

In [22]:
#0 A boy wearing a red shirt and jeans is doing a flip on his bike.
#1 A person flipping a bicycle upside down.
#2 a person flips on a bike.
#3 A person in a red shirt doing tricks on a bicycle .
#4 A person is show upside down on his bicycle over a large field .

#0	A group of people are walking while holding numerous baskets and rugs .
#1	A man carrying many woven baskets next to a man carrying a stack of rugs on his back .
#2	Two individuals carrying numerous products to sell are walking on a beach .
#3	Two people carrying colorful baskets and blankets walk near a building .
#4	Two people walking laden down with baskets and blankets .

#0	A man in black is sitting next to a modern art structure in front of a glass building .
#1	A man sits and reads a newspaper by a sculpture outside of an office building .
#2	a man sits near a large statue .
#3	A man sitting in front of a metal sculpture in front of a building .
#4	The man with the backpack is sitting in a buildings courtyard in front of an art sculpture reading.

#0	A crowd watching air balloons at night .
#1	A group of hot air balloons lit up at night .
#2	People are watching hot air balloons in the park .
#3	People watching hot air balloons .
#4	Seven large balloons are lined up at nighttime near a crowd .

#0	A cyclist is riding a bicycle on a curved road up a hill .
#1	A man in aerodynamic gear riding a bicycle down a road around a sharp curve .
#2	A man on a mountain bike is pedaling up a hill .
#3	Man bicycle up a road , while cows graze on a hill nearby .
#4	The biker is riding around a curve in the road .

In [23]:
cnet = Convolutional_Encoder(30,10,20)
for img,cap in loader:
    print(cnet(img))
    break

XOUT tensor([[[[31.6706, 28.4788, 16.9788,  ..., 24.4377, 35.2012, 35.8153],
          [29.3412, 27.0988, 15.4224,  ..., 22.9988, 27.4235, 27.6530],
          [26.5824, 24.0600, 13.9447,  ..., 23.7471, 27.1730, 26.8083],
          ...,
          [40.3059, 22.5647, 14.9871,  ...,  9.1330,  9.4353,  9.6612],
          [48.2036, 42.0836, 35.5612,  ..., 16.2741, 14.5271, 14.1271],
          [52.5318, 52.9283, 49.4012,  ..., 18.6930, 18.1377, 15.2400]],

         [[31.6676, 28.4758, 16.9759,  ..., 24.4347, 35.1982, 35.8123],
          [29.3382, 27.0958, 15.4194,  ..., 22.9959, 27.4206, 27.6500],
          [26.5794, 24.0570, 13.9417,  ..., 23.7441, 27.1700, 26.8053],
          ...,
          [40.3029, 22.5617, 14.9841,  ...,  9.1300,  9.4323,  9.6582],
          [48.2006, 42.0806, 35.5582,  ..., 16.2712, 14.5241, 14.1241],
          [52.5288, 52.9253, 49.3982,  ..., 18.6900, 18.1347, 15.2370]],

         [[31.6626, 28.4708, 16.9708,  ..., 24.4297, 35.1932, 35.8073],
          [29.3332, 27.09